<a href="https://colab.research.google.com/github/meghana0227/YouTube-Video-Summarizer/blob/main/Youtube_Video_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Audio Extraction**

In [ ]:
pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.7 MB/s eta 0:00:00


In [ ]:
import yt_dlp

def download_audio(url, filename='audio'):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': filename,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print(f"Audio downloaded as {filename}")


download_audio("https://www.youtube.com/watch?v=n434ha4QwU0&ab_channel=EnglishSpeeches")


[youtube] Extracting URL: https://www.youtube.com/watch?v=n434ha4QwU0&ab_channel=EnglishSpeeches
[youtube] n434ha4QwU0: Downloading webpage
[youtube] n434ha4QwU0: Downloading ios player API JSON
[youtube] n434ha4QwU0: Downloading mweb player API JSON
[youtube] n434ha4QwU0: Downloading player baafab19
[youtube] n434ha4QwU0: Downloading m3u8 information
[info] n434ha4QwU0: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of   18.82MiB in 00:00:00 at 24.06MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio (pass -k to keep)
Audio downloaded as audio


**Audio Transcription using Whisper**

In [ ]:
pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-mw527p8o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-mw527p8o
  Resolved https://github.com/openai/whisper.git to commit 271445b2f24f00f8175c4fb7ae91876f7451dfc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803557 sha256=380fb46a06e64bd08fe9caa12fc580d26a49424761681c3d680fab1c5a8a02cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-alutl5ox/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
import whisper

def transcribe_audio(filename='audio.mp3'):
    model = whisper.load_model("base")
    result = model.transcribe(filename)
    transcript = result['text']
    print("Transcription Complete")
    return transcript

transcript = transcribe_audio('audio.mp3')
print(transcript)


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 83.0MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription Complete
 The future depends on all of us. And whether you like Facebook or not, I think we need to recognize what is at stake and come together to stand for voice and free expression at this critical moment. It's really great to be here at Georgetown with all of you today. Before we get started, I just want to take a moment to acknowledge that today we lost an icon, Elijah Cummings. He was a powerful voice for equality and for social progress and for bringing people together. Back when when I was in college, our country had just gone to war in Iraq. And the mood on our campus was disbelief. A lot of people felt like we were acting without hearing a lot of important perspectives. And the toll on soldiers and their families and our national psyche was severe. Yet most of us felt like we were powerless to do anything about it. And I remember feeling that if more people had a voice to share their experiences, then maybe it could have gone differently. And those early years s

**Summarization using PEGASUS**

In [ ]:
pip install transformers


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch


model_name = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def chunk_text(text, max_tokens=512):
    """Splits text into chunks that fit within the model's token limit."""
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunk = " ".join(words[i:i + max_tokens])
        chunks.append(chunk)
    return chunks

def summarize_long_text(text, max_length=150, min_length=50):
    chunks = chunk_text(text, max_tokens=tokenizer.model_max_length - 10)
    chunk_summaries = []

    for chunk in chunks:
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True)
        with torch.no_grad():
            summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, no_repeat_ngram_size=2)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        chunk_summaries.append(summary)


    combined_summary = " ".join(chunk_summaries)
    return combined_summary

summary = summarize_long_text(transcript)
print("Final Summary:", summary)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:  38%|###7      | 860M/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

Final Summary: Here is the full text of Mark Zuckerberg's speech at Georgetown University on Wednesday, in which he called for more people to have a "voice" and "bring people together": Here in America, we're at a moment of great tension. In our series of letters from African-American journalists, filmmaker and columnist Richard Pérez-Pea looks at how free speech has been central to the fight for democracy around the world and how it's become much broader in the United States. In our series of letters from African-American journalists, Supreme Court Justice Ruth Bader Ginsburg reflects on the importance of free expression and the role it plays in our society. But there's also an important counter trend, which is that in times of social tension, our impulse is often to pull back on free Expression because we want the progress that comes from freeExpression, but we don't Want the Tension. Here is the full text of Mark Zuckerberg's speech at the World Economic Forum in Davos, Switzerland,